In [10]:
from muselsl import *
import time
import numpy as np
import serial
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import json
import warnings
from command import Tracker, SHIFT_LENGTH, NUM_CHANNELS, EPOCH_LENGTH, OVERLAP_LENGTH, BUFFER_LENGTH, Band
import xml.dom.minidom

In [30]:

# -*- coding: utf-8 -*-
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data
import utils  # Our own utility functions
from process_data import ChannelDataProcessor, Metrics
from PsychoPy_Code.BrainWave import run_psychopy

# Handy little enum to make code more readable



class Channel:
    TP0 = 0
    FP1 = 1
    FP2 = 2
    TP10 = 3
    DRL = 4



def _acquire_eeg_data_mock():
    """ Get eeg_data and timestamp from json for testing purposes. 

    :return: tuple: _eeg_data, _timestamp
    """
    json_file = open("to_send.json")
    k = json.loads(json_file.read())
    _eeg_data = k[0]["eeg_data"]
    _timestamp = k[0]["timestamp"]
    return _eeg_data, _timestamp

if __name__ == "__main__":

    """ 
    1. CONNECT TO EEG STREAM 
    
    """
     
    # Search for active LSL streams
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if len(streams) == 0:
        raise RuntimeError('Can\'t find EEG stream.')
    
    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()
    
    # Get the stream info and description
    info = inlet.info()
    description = info.desc()
    
    """
    2. Obtain Threshold from PsycoPy 
    
    """
    
    cmd = Tracker(inlet=inlet, info=info, channel_index=Channel.FP1)
    
    cmd.start_stage(mode='relax', stage=1)
    
    time.sleep(1)
    
    cmd.end_stage(mode='relax', stage=1)
    print(EPOCH_LENGTH)
    #run_psychopy(cmd=cmd)
    #cmd.update_stage_threshold()
    threshold = 0.0223716588909309062
    print("FINAL THRESH", threshold)

    """
    3. Record data and wave hand if metric is above threshold.  
    
    """
    
    # The try/except structure allows to quit the while loop by aborting the
    # script with <Ctrl-C>
    print('Press Ctrl-C in the console to break the while loop.')
    
    
    def get_fs(_info):
        return int(_info.nominal_srate())

    fs = get_fs(info)

    def _acquire_eeg_data(_inlet):
        """ Pull EEG data from inlet and return.

        :return: tuple: _eeg_data, _timestamp
        """
        _eeg_data, _timestamp = _inlet.pull_chunk(
            timeout=1, max_samples=int(SHIFT_LENGTH * fs))
        return _eeg_data, _timestamp

    arduinoData = serial.Serial('com3',9600)
    time.sleep(1)
    try:
        
        
        # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
        while True:
            #
            """ 3.1 ACQUIRE DATA """

            # Obtain EEG data from the LSL stream
            eeg_data, timestamp = _acquire_eeg_data(inlet)           
            #print(eeg_data)
            #print(" ")
            c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH, epoch_length=EPOCH_LENGTH,
                                 overlap_length=OVERLAP_LENGTH, 
                                 shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)
        
            
            c.feed_new_data(eeg_data=eeg_data)  # Feed new data generated in the epoch
        
            metrics = []

            for i in range(NUM_CHANNELS):  # Iterate through all separate channels

                # Record channel smooth band power
                csbp = c.get_channel_smooth_band_powers(i) 
                # Run calculations on csbp to obtain desired metrics
                #print(csbp)
                if i in [1,2,3]:
                    theta_metric = csbp[1]
                
                    metrics.append(theta_metric)
                
            metric = np.mean(metrics)  # TODO: switch to the correct channel  
            print(metric)    
            def check_positive(_metric, _threshold):
                """
                Returns True if beta_metric is less than the threshold 
                :return: 
                """
                return _metric > _threshold
            
            def hand_wave():
                
                action = '1'    #pass in the input of either 1 or 0
                
                arduinoData.write(action.encode())
            
            def hand_stay():
                
                action = '0'
                
                arduinoData.write(action.encode())
            
            if check_positive(_metric=metric, _threshold=threshold): 
                
                hand_wave()
            else:
                hand_stay()
            
    except KeyboardInterrupt:
        print('Closing!')   
    arduinoData.close()

Looking for an EEG stream...
Start acquiring data
Press Ctrl-C in the console to break the while loop.
average theta: -0.008343448005773238
1
FINAL THRESH 0.022371658890930905
Press Ctrl-C in the console to break the while loop.
0.014633389422591285
0.01338472794619589
0.016039604116436665
0.006460716110012542
0.01295858131172313
0.02145997678823139
0.011602710056115718
0.011366588549278711
0.012357940510468091
0.01451573148913101
0.023624240775407906
0.03279646225638827
0.03297259882876688
0.040404586941188425
0.03878028993649774
0.03547074086089658
0.016535151213982528
0.019535003957128937
0.01579924028701862
0.038783655939272535
0.04191602641707695
0.03829239632436066
0.040164424575244384
0.04189875745277608
0.04263566619289083
0.034804545330720346
0.030417896144749917
0.01897567017511016
0.012616116277061808
0.026551680076496772
0.036805943459783434
0.0392702984536654
0.03532965595026564
0.04362609602991347
0.03482823080206834
0.04824586154134294
0.038317976442996435
0.042662500812

0.02788583419542438
0.025092554359654
0.03278222224885017
0.034106902330450835
0.031040244461403663
0.031271652824618267
0.0412771468580309
0.03959910640278182
0.03856433798772737
0.035643085188133576
0.03939078913599634
0.041948972137842745
0.03898558990933412
0.042023004042429306
0.029827245032897003
0.028564740002014078
0.041652790999490624
0.035029159513949686
0.038940968232720655
0.0294238476291752
0.029555988584228626
0.025987029643691737
0.0348934863879394
0.03134781468158876
0.029392001785024255
0.035358641751771024
0.026794842038345756
0.027114601915878667
0.03442484672959862
0.028166531878169077
0.02669178759140432
0.030622519874760654
0.0222303172400124
0.03906131802319762
0.04528655297668654
0.03062577578822781
0.023049380253564525
0.027349649135247723
0.023519034014259147
0.028374627958805878
0.022133088260279234
0.03281841001830226
0.028545486879546656
0.029092280038041513
0.029637130434328196
0.024759688363274284
0.02613000592696269
0.021896077593174985
0.031230979782293

0.01564505638006462
0.010911484198270057
0.021458626067310527
0.014620798250039405
-0.0013322712981125053
0.009572309711867144
0.009550585145026257
0.012103666090160677
0.01121639329626127
0.01047215329233462
0.012520332114827388
0.02054665805702355
0.011598071593235842
0.007255195380584364
0.002388478075449736
0.014526091927478493
0.013174071720523692
0.012970302000469884
0.015741305448701108
0.02059548738968228
0.01427160200688543
0.010029255456540653
0.01713162364831427
0.014085319820725
0.01000581441371824
0.0255674052784764
0.01292485617706429
0.010277455831313053
0.015230570234941952
0.007532632622492333
0.009691196868654993
0.006422030988621177
0.008187387828552188
0.013446626366701186
0.006465859350519226
0.01639876651925783
0.031808570138140785
0.019192521219886488
0.021715734992829226
0.015466954911600267
0.014113890341935722
0.012478354130836372
-0.006413073687707831
0.009279705976426212
0.00897342944299989
0.010858300936726606
0.00845340704748612
0.006414177959256754
0.0108

In [27]:
arduinoData = serial.Serial('com3',9600)
arduinoData.write ('0'.encode('UTF-8'))
arduinoData.close()

In [28]:
arduinoData = serial.Serial('com3',9600)
action = '1'    #pass in the input of either 1 or 0
arduinoData.write(action.encode())

arduinoData.close()

In [76]:
(0.10679642 + 0.07821495 + 0.09640047 +0.09640047 + 0.10925018)/5

0.097412498